In [6]:
import cutlass
import cutlass.cute as cute
import torch
from cutlass.torch import dtype as torch_dtype

In [11]:
@cute.jit
def test(ptr: cute.Pointer):
    layout = cute.make_layout(shape=((2,2),2), stride=((4,1),2))
    tensor = cute.make_tensor(ptr, layout)
    cute.print_tensor(tensor, verbose=False)
    cute.print_tensor(tensor, verbose=True)

dtype = cutlass.Int32
storage = torch.arange(8, dtype=torch_dtype(dtype))
storage_ptr = cute.runtime.make_ptr(dtype, storage.data_ptr())
test(storage_ptr)

tensor(raw_ptr(0x0000556bce23dc00: i32, generic, align<4>) o ((2,2),2):((4,1),2), data=
       [[ 0,  2, ],
        [ 4,  6, ],
        [ 1,  3, ],
        [ 5,  7, ]])
tensor(raw_ptr(0x0000556bce23dc00: i32, generic, align<4>) o ((2,2),2):((4,1),2), data= (
	((0,0),0)= 0
	((0,1),0)= 1
	((0,0),1)= 2
	((0,1),1)= 3
	((1,0),0)= 4
	((1,1),0)= 5
	((1,0),1)= 6
	((1,1),1)= 7
)
